<a href="https://colab.research.google.com/github/wenko99/Standalone_DDL/blob/master/Lec09/Lec09_Simpsons_Model_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Grading Simpsons Models

<br/>

Google Drive의 **Simpsons** 폴더 안에 **simpsons_models**라는 폴더를 새로 만드시고,

그 안에 Model의 정보가 담긴 **.h5, .json** 파일들을 업로드해주세요.

또, **simpsons_testset.zip** 파일을 받아 압축을 푸시고, 나오는 **simpsons_testset** 폴더를 그대로 **Simpsons** 폴더 안에 업로드해주세요.

<br/>

## Download and Preprocess Test Data

###Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
cd drive/My Drive/Simpsons

In [0]:
ls

##Preprocess Simpsons Dataset

In [0]:
import os
import numpy as np
import skimage as im

In [0]:
characters = ['abraham_grampa_simpson', 'maggie_simpson', 'bart_simpson', 'lisa_simpson', 'marge_simpson', 'homer_simpson']

**JPG to CSV**

<br/>

이미 csv 파일을 제공해 드렸으니, 아래 코드 셀은 실행하지 않으셔도 됩니다.

<br/>

In [0]:
dirname = './simpsons_testset/kaggle_simpson_testset/'
data = [[0 for i in range(1)] for j in range(6)]
for filename in os.listdir(dirname):
    if filename.endswith('.jpg'):
        for i in range(len(characters)):
            if characters[i] in filename:
                path = os.path.join(dirname, filename)
                img = im.io.imread(path)
                img = np.array(im.transform.resize(img, (64, 64, 3)))
                img = img.flatten()
                img = np.append(i, img)
                data[i].append(img)
                break
dirname = './simpsons_testset/'
for i in range(len(data)):
    data[i].pop(0)
    with open(dirname + characters[i] + '_test.csv', 'wb') as f:
        print('Data saved onto {loc} with len {len}'.format(loc=dirname+characters[i]+'_test.csv', len=len(data[i])))
        np.savetxt(f, data[i], delimiter=',')

###Open CSV

In [0]:
import pandas as pd

In [0]:
frames = []

dirname = './simpsons_testset/'
ext = '_test.csv'
bookmark = 0
for i in range(len(characters)):
    path = dirname + characters[i] + ext
    print(path)
    frames.append(pd.read_csv(path, header=None))

# Concatenate all Dataframes
df = pd.concat(frames, ignore_index=True)

# Shuffle Dataset
np.random.seed(7)
df = df.reindex(np.random.permutation(df.index))

# Separate into Feature / Label
test_data = df.iloc[:,1 : ].values
test_labels = df.iloc[:, 0].values.astype(int)

# Reshape Vector into Tensor
test_data = test_data.reshape((test_data.shape[0], 64, 64, -1))

In [0]:
# Check Shape
print('Test Data Shape : {}'.format(str(test_data.shape)))
print('Test Label Shape : {}'.format(str(test_labels.shape)))

##Visualize Test Data

In [0]:
import matplotlib.pyplot as plt

In [0]:
def visualize(data, labels, idx=0):
    fig = plt.figure()
    for i in range(6):
        ax = fig.add_subplot(2, 3, i + 1)
        image = data[idx]
        ax.imshow(image)
        ax.set_title(characters[labels[idx]][:-8])
        idx += 1
    plt.subplots_adjust(hspace = 0.5, wspace = 0.3)
    plt.show()

    print("The shape of the image is {}".format(image.shape))

visualize(test_data, test_labels, 0)

## Download Model Architecture / Parameters

In [0]:
import tensorflow as tf
import os

In [0]:
models = []
model_names = []

for filename in os.listdir('./simpsons_models'):
    if filename.endswith('.json'):
        path = os.path.join('./simpsons_models', filename)
        print(path)
        with open(path, 'r') as f:
            model_tmp = tf.keras.models.model_from_json(f.read())
        path = path[:-17]
        path = path + 'weights.h5'
        model_tmp.load_weights(path)
        models.append(model_tmp)
        model_names.append(path[18:-11])
        continue
    else:
        continue

In [0]:
model_names

##Test Accuracy

In [0]:
# Reshape Data so that Conv2D is applicable
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], test_data.shape[2], -1))

In [0]:
def test(model):
    prediction = np.argmax(model.predict(test_data), 1)

    acc = 0
    for i in range(len(test_data)):
        if prediction[i] == test_labels[i]:
            acc += 1
    acc /= len(test_data)

    print('Accuracy over Test Set is {}%'.format(round(acc, 5) * 100))
    
    return round(acc, 5) * 100

    #test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], -1))

    #visualize(test_data, prediction)

In [0]:
res = {}

for i in range(len(models)):
    print('\nModel : {}\n'.format(model_names[i]))
    res.update({model_names[i] : test(models[i])})

In [0]:
import operator

res = sorted(res.items(), key=operator.itemgetter(1), reverse=True)

print("LEADERBOARD\n")
for i in range(len(res)):
    print(res[i])